In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size" : 50257,
    "context_length" : 256,
    "emb_dim" : 768,
    "n_heads" : 12,
    "n_layers" : 12,
    "drop_rate" : 0.1,
    "qkv_bias" : False
}

In [3]:
class LayerNorm(nn.Module):

    def __init__(self, emb_dim):
        super().__init__()

        self.eps = 1e-05
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):

        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x-mean) / (torch.sqrt(variance + self.eps))

        return self.scale * norm_x + self.shift
    
class GELU(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        return  0.5 * x * ( 1 + torch.tanh( torch.sqrt(  torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3)  ) ))

class FeedForward(nn.Module):

    def __init__(self, cfg):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4*cfg["emb_dim"]),
            GELU(),
            nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"])
        )

    def forward(self, x):
        return self.layers(x)
    
class MultiHeadAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, num_heads, dropout, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)

        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
    
    def forward(self, x):

        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

In [4]:
class TransformerBlock(nn.Module):

    def __init__(self, cfg):
        super().__init__()

        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
        self.ff = FeedForward(cfg=cfg)
    
    def forward(self, x):

        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

In [5]:
torch.manual_seed(123)

sample_input = torch.rand(2, 4, 768)
tb = TransformerBlock(GPT_CONFIG_124M)

print("Shape of input to transformer block: ", sample_input.shape)
output = tb(sample_input)
print("Shape of output of transformer block: ", output.shape)

Shape of input to transformer block:  torch.Size([2, 4, 768])
Shape of output of transformer block:  torch.Size([2, 4, 768])


In [6]:
class GPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()

        self.token_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[ TransformerBlock(cfg) for _ in range(cfg["n_layers"]) ]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])

        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.token_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

In [7]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [8]:
torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)

model.eval()

GPTModel(
  (token_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_featu

In [9]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]
        probab = torch.softmax(logits, dim=-1)
        id_next = torch.argmax(probab, dim=-1, keepdim=True)
        idx = torch.cat((idx, id_next), dim=1)
    return idx

In [10]:
# This is a code that generates next max_new_tokens number of new tokens to extend the initial text

def text_to_token_ids(text, tokenizer):
    tokens = tokenizer.encode(text, allowed_special={"<|endoftoken|>"})
    tokens = torch.tensor(tokens).unsqueeze(0)
    return tokens

def token_ids_to_text(idx, tokenizer):
    text = idx.squeeze(0)
    return tokenizer.decode(text.tolist())

start_context = "Every effort moves"
gtt = generate_text_simple(model=model, idx=text_to_token_ids(start_context, tokenizer=tokenizer), max_new_tokens=10, context_size=GPT_CONFIG_124M["context_length"])
print(gtt)
print("Generated text: ", token_ids_to_text(gtt, tokenizer=tokenizer))

tensor([[ 6109,  3626,  6100, 20610, 22213, 15924,  9763,  3519, 20495, 36563,
         39382, 39539, 40415]])
Generated text:  Every effort movesFridaycritRober backed related starring]); detects estimatingGGGGGGGG


In [11]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [12]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1)
print("Probabilities shape: ", probas.shape)

Probabilities shape:  torch.Size([2, 3, 50257])


In [13]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Output:")
print(token_ids)

Output:
tensor([[[36397],
         [39619],
         [20610]],

        [[ 8615],
         [49289],
         [47105]]])


In [14]:
print("Generated text: ")
print(token_ids_to_text(token_ids[0].flatten(), tokenizer=tokenizer))
print("Required text:")
print(token_ids_to_text(targets[0], tokenizer=tokenizer))

Generated text: 
 Gathering SerbianFriday
Required text:
 effort moves you


In [15]:
# Finding loss from logits (output of gpt model) and target

print("Shape of probability output tensor: ", probas.shape)

print("Shape of target: ", targets.shape)

target_probabilities = []
for b in range(probas.shape[0]):
    batc = b
    target_probabilities.append(probas[batc, torch.arange(probas.shape[1]), targets[batc]])
print("Shape of requre: ", target_probabilities)
target_probabilities = torch.cat(target_probabilities, dim=0)
print(target_probabilities)

target_probabilities = torch.log(target_probabilities)
print(target_probabilities)

loss = torch.mean(target_probabilities)
print(loss)
loss = (loss * -1)
print("Loss: ", loss)

Shape of probability output tensor:  torch.Size([2, 3, 50257])
Shape of target:  torch.Size([2, 3])
Shape of requre:  [tensor([2.3466e-05, 2.0531e-05, 1.1733e-05]), tensor([4.2794e-05, 1.6248e-05, 1.1586e-05])]
tensor([2.3466e-05, 2.0531e-05, 1.1733e-05, 4.2794e-05, 1.6248e-05, 1.1586e-05])
tensor([-10.6600, -10.7936, -11.3531, -10.0591, -11.0276, -11.3658])
tensor(-10.8765)
Loss:  tensor(10.8765)


In [16]:
logits_flaten = logits.flatten(0, 1)
target_flaten = targets.flatten(0, 1)
print("shape of logits and target after flatten:")
print(logits_flaten.shape)
print(target_flaten.shape)

shape of logits and target after flatten:
torch.Size([6, 50257])
torch.Size([6])


In [17]:
loss = torch.nn.functional.cross_entropy(logits_flaten, target_flaten)
print(loss)

tensor(10.8765)


In [18]:
perplexity = torch.exp(loss)
print(perplexity)

tensor(52918.7773)


In [19]:
# torch.manual_seed(123)

# model = GPTModel(GPT_CONFIG_124M)

# print("Shape of input: ", batch.shape)

# output = model(batch)

# print("Shape of output: ", output.shape)

In [20]:
# Total_parameters = sum(p.numel() for p in model.parameters())
# print("Total number of parameters in GPT model: ", Total_parameters)

In [21]:
# print("Token embedding layer shape: ", model.token_emb.weight.shape)
# print("Output layer shape: ", model.out_head.weight.shape)

In [22]:
# Total_parameters_gpt2 = Total_parameters - sum(p.numel() for p in model.out_head.parameters())
# print("Total parameters using weight tying: ", Total_parameters_gpt2)

In [23]:
# # To calculate total space requirement

# total_param_bytes = Total_parameters * 4
# total_param_mb = total_param_bytes / (1024*1024)
# print(f"Total memory requirement: {total_param_mb:.2f} MB")

In [24]:
# def generate_text_simple(model, idx, max_new_tokens, context_size):
#     for _ in range(max_new_tokens):
#         idx_cond = idx[:, -context_size:]
        
#         # Get the predictions
#         with torch.no_grad():
#             logits = model(idx_cond)

#         logits = logits[:, -1, :]
#         probab = torch.softmax(logits, dim=-1)
#         id_next = torch.argmax(probab, dim=-1, keepdim=True)
#         idx = torch.cat((idx, id_next), dim=1)
#     return idx

In [25]:
# start_context = "Hello, I am"
# encoded = tokenizer.encode(start_context)
# print("encoded:", encoded)
# encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
# print("encoded_tensor.shape:", encoded_tensor.shape)

In [26]:
# model.eval()

# out = generate_text_simple(
#     model=model,
#     idx=encoded_tensor,
#     max_new_tokens=6,
#     context_size=GPT_CONFIG_124M["context_length"]
# )
# print("Output:", out)
# print("Output length:", len(out[0]))

In [27]:
# print(tokenizer.decode(out.squeeze(0).tolist()))

In [28]:
with open("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/NLP_LLM/the-verdict.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

print("Total number of charecters in the text: ", len(text_data))
print("\nFirst 100 charecters of raw_text:\n", text_data[:99])
print("\nLast 100 charecters of text_data: \n", text_data[-99:])

Total number of charecters in the text:  20479

First 100 charecters of raw_text:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 

Last 100 charecters of text_data: 
 it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


In [29]:
tokenizer = tiktoken.get_encoding("gpt2")

In [30]:
total_charecters = len(text_data)
tokens = tokenizer.encode(text=text_data)
total_tokens = len(tokens)

print("Total charecters in text data: ", total_charecters)
print("Total tokens after tokenization: ", total_tokens)

Total charecters in text data:  20479
Total tokens after tokenization:  5145


In [31]:
from torch.utils.data import DataLoader

In [32]:
class GPTDatasetV1:
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i : i+max_length]
            ouput_chunk = token_ids[i+1 : i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(ouput_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.output_ids[idx]
    
def create_dataloader_v1(txt, max_length, stride, batch_size, shuffle, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length=max_length, stride=stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        shuffle=shuffle,
        num_workers=num_workers
    )
    return dataloader

In [33]:
train_ratio = 0.9
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    txt=train_data,
    stride=GPT_CONFIG_124M["context_length"],
    max_length=GPT_CONFIG_124M["context_length"],
    batch_size=2,
    shuffle=True,
    drop_last=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    txt=val_data,
    stride=GPT_CONFIG_124M["context_length"],
    max_length=GPT_CONFIG_124M["context_length"],
    batch_size=2,
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [34]:
if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

Not enough tokens for the validation loader. Try to lower the `GPT_CONFIG_124M['context_length']` or decrease the `training_ratio`


In [35]:
print("Training loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValid loader")
for x, y in val_loader:
    print(x.shape, y.shape)

Training loader:
torch.Size([2, 1024]) torch.Size([2, 1024])
torch.Size([2, 1024]) torch.Size([2, 1024])

Valid loader


In [36]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

valid_tokens = 0
for input_batch, target_batch in val_loader:
    valid_tokens += input_batch.numel()

print("Train tokens: ", train_tokens)
print("Validation tokens: ", valid_tokens)
print("Total number of tokens: ", train_tokens + valid_tokens)

Train tokens:  4096
Validation tokens:  0
Total number of tokens:  4096


In [37]:
class GPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()

        self.token_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[ TransformerBlock(cfg) for _ in range(cfg["n_layers"]) ]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])

        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.token_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

In [38]:
torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (token_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_featu

In [39]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

def calc_loss_loader(dataloader, model, device, num_batches=None):
    total_loss = 0.
    if len(dataloader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(dataloader)
    else:
        num_batches = min(num_batches, len(dataloader))
    
    for i, (input_batch, target_batch) in enumerate(dataloader):
        loss = calc_loss_batch(input_batch=input_batch, target_batch=target_batch, model=model, device=device)
        total_loss += loss.item()
    
    return total_loss / num_batches

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device=device)

torch.manual_seed(123)

with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)


print("Training loss: ", train_loss)
print("Validation loss: ", val_loss)

Training loss:  10.999300956726074
Validation loss:  nan


In [41]:
# print(device)